## Test of Cell Tracking Application

In [1]:
from deepcell.applications import NuclearSegmentation, CytoplasmSegmentation, CellTracking
from deepcell.datasets import DynamicNuclearNetSample

import numpy as np
import tifffile
import imageio
import copy
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

## Load Data
1. Dynamic Nuclear Fluorescence
2. Dynamic Collagen and Cytoplasm multichannel

In [2]:
x, y, _ = DynamicNuclearNetSample().load_data()
cell_data = tifffile.imread('cell_track_data/12.20.17_combo2-cell1.tif')

print(f"caliban data shape: {x.shape}, prediction shape: {y.shape}")
print(f"collagen data shape: {cell_data.shape}")

caliban data shape: (10, 540, 540, 1), prediction shape: (10, 540, 540, 1)
collagen data shape: (57, 3, 2, 210, 208)


### Reshape collagen data

DeepCell does not yet work with 3D segmentation, so we must pick one of the three z-slices.

We also notice that the shape of DeepCell data is (T, X, Y, C) whereas our data is in (T, Z, C, X, Y) so we must reshape

In [3]:
# Shuffle dimensions
cell_data = np.transpose(cell_data, (0,3,4,2,1))
print(f'after tranposing: {cell_data.shape}')

# Take one z-slice
z_pos = 2
cell_data = cell_data[..., z_pos]
print(f'single z-slice: {cell_data.shape}')

# Separate collagen and cytoplasm channel
collagen_data = np.expand_dims(cell_data[..., 0], 3)
cytoplasm_data = np.expand_dims(cell_data[..., 1], 3)
print(f'collagen shape: {collagen_data.shape}\ncytoplasm shape: {cytoplasm_data.shape}')

after tranposing: (57, 210, 208, 2, 3)
single z-slice: (57, 210, 208, 2)
collagen shape: (57, 210, 208, 1)
cytoplasm shape: (57, 210, 208, 1)


### Save images as gifs

In [40]:
def save_gif(x, filename, title):
    # ims = [Image.fromarray(x[i,...,0], 'LA') for i in range(x.shape[0])]
    # ims[0].save(filename, save_all=True, 
    #             append_images=ims[1:],
    #             optimize=False, loop=0 )
    
    def plot(im):
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(im, 'Greys_r', vmax=3000)
        plt.axis('off')
        plt.title(title)

        fig.canvas.draw()  # draw the canvas, cache the renderer
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

        plt.close(fig)

        return image

    imageio.mimsave(filename, [plot(x[i, ..., 0]) for i in range(x.shape[0])], loop=0)
    


save_gif(x, 'caliban-raw.gif', 'Caliban')
save_gif(cytoplasm_data, f'12.20.17_combo2-cell1-cyto-zpos{z_pos}.gif', 'Cytoplasm')

### Display saved 

<img src="caliban-raw.gif" width="300pt" align="center" loop="infinite">
<img src="12.20.17_combo2-cell1-cyto-zpos2.gif" width="300pt" align="center" loop="infinite">

## Methods for displaying models

In [4]:
def shuffle_colors(ymax, cmap):
    """Utility function to generate a colormap for a labeled image"""
    cmap = mpl.colormaps[cmap].resampled(ymax)
    nmap = cmap(range(ymax))
    np.random.shuffle(nmap)
    cmap = ListedColormap(nmap)
    cmap.set_bad('black')
    return cmap

In [6]:
def save_with_seg(x, y, filename):
    ymax = np.max(y)
    cmap = shuffle_colors(ymax, 'tab20')

    def plot(x, y):
        yy = copy.deepcopy(y)
        yy = np.ma.masked_equal(yy, 0)

        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(x, cmap='Greys_r', vmax=3000)
        ax[0].axis('off')
        ax[0].set_title('Raw')
        ax[1].imshow(yy, cmap=cmap, vmax=ymax)
        ax[1].set_title('Segmented')
        ax[1].axis('off')

        fig.canvas.draw()  # draw the canvas, cache the renderer
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        plt.close(fig)

        return image

    imageio.mimsave(
        filename,
        [plot(x[i,...,0], y[i,...,0])
        for i in range(y.shape[0])],
        loop=0
    )


## Run Segmentation model on data to create masks

Before any cell tracking can happen, DeepCell requires masks to be created by its segmentation models.

In this section, we will also create stardist and cellpose masks to see if there is any overlap in compatibility.

In [41]:
# DeepCell on caliban nuclear
deep_nuclear = NuclearSegmentation()
y_pred = deep_nuclear.predict(x, image_mpp=0.65)
save_with_seg(x, y_pred, 'caliban-segmented.gif')

INFO:root:Checking for cached data
INFO:root:Checking NuclearSegmentation-75.tar.gz against provided file_hash...
INFO:root:NuclearSegmentation-75.tar.gz with hash efc4881db5bac23219b62486a4d877b3 already available.
INFO:root:Extracting C:\Users\aarus\.deepcell\models\NuclearSegmentation-75.tar.gz
INFO:root:Successfully extracted C:\Users\aarus\.deepcell\models\NuclearSegmentation-75.tar.gz into C:\Users\aarus\.deepcell\models


INFO:root:Converting image dtype to float


INFO:root:Converting image dtype to float


In [47]:
# DeepCell on example cytoplasm data
deep_cyto = CytoplasmSegmentation()
deep_cyto_pred = deep_cyto.predict(cytoplasm_data, image_mpp=0.5)
save_with_seg(cytoplasm_data, deep_cyto_pred, '12.20.17_combo2-cell1-cyto-zpos2-segmented.gif')

INFO:root:Converting image dtype to float


In [7]:
# Stardist on example cytoplasm data
from stardist.models import StarDist2D
from csbdeep.utils import normalize

sd_model = StarDist2D.from_pretrained('2D_versatile_fluo')
labels = [sd_model.predict_instances(normalize(item))[0] for item in cytoplasm_data]
sd_predict = np.expand_dims(np.array(labels), 3)
save_with_seg(cytoplasm_data, sd_predict, 'stardist_seg.gif')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [15]:
# Cellpose on example cytoplasm data
from cellpose import models

cyto3 = models.Cellpose(model_type='cyto3')
labels = [cyto3.eval(x, channels=[0,0])[0] for x in cytoplasm_data]
cellpose_predict = np.expand_dims(np.array(labels), 3)
save_with_seg(cytoplasm_data, cellpose_predict, 'cellpose_seg.gif')